In [1]:
import pandas as pd
import re
from rapidfuzz import process, fuzz
import numpy as np


def _norm(s: str) -> str:
    s = str(s).lower().strip()
    s = re.sub(r"[^\w\s'-]", " ", s)
    s = s.replace("-", " ")
    s = re.sub(r"\s+", " ", s)
    return s

def _parse_name(s: str):
    s = _norm(s)
    parts = s.split()
    if not parts:
        return {"first":"", "middles":[], "last":"", "tokens":set()}
    prefixes = {"dr","mr","mrs","ms","miss"}
    suffixes = {"jr","sr","ii","iii","iv"}
    parts = [p for p in parts if p not in prefixes|suffixes]
    if not parts:
        return {"first":"", "middles":[], "last":"", "tokens":set()}
    first = parts[0]
    last = parts[-1] if len(parts) > 1 else ""
    middles = parts[1:-1] if len(parts) > 2 else []
    return {"first": first, "middles": middles, "last": last, "tokens": set(parts)}

def _first_match_score(a_first: str, b_first: str) -> float:
    if not a_first or not b_first:
        return 0.0
    a_first = a_first.strip().lower()
    b_first = b_first.strip().lower()
    # initial vs full name
    if a_first[0] == b_first[0] and (len(a_first) == 1 or len(b_first) == 1):
        return 95.0
    # fallback similarity
    return float(fuzz.ratio(a_first, b_first))

def _last_match_score(a_last: str, b_last: str) -> float:
    if not a_last or not b_last:
        return 0.0
    a = a_last.replace("'", "")
    b = b_last.replace("'", "")
    return float(max(fuzz.ratio(a, b), fuzz.token_sort_ratio(a, b)))

def name_similarity(a: str, b: str, *, score_cutoff: float = 0.0) -> float:
    pa, pb = _parse_name(a), _parse_name(b)
    last = _last_match_score(pa["last"], pb["last"])
    if last < 75:
        return 0.0
    first = _first_match_score(pa["first"], pb["first"])
    middle = 0.0
    if pa["middles"] and pb["middles"]:
        middle = float(fuzz.token_set_ratio(" ".join(pa["middles"]), " ".join(pb["middles"])))
    full = float(fuzz.token_set_ratio(_norm(a), _norm(b)))
    weighted = 0.7*last + 0.25*first + 0.05*middle
    score = max(weighted, full if last >= 85 else weighted)
    return score if score >= score_cutoff else 0.0

def match_providers(providers, org_providers, threshold=76):
    matched = []
    unmatched_map = {}  # providers_norm -> best unmatched

    matched_providers_set = set()  # to avoid re-matching same provider
    already_matched_org_providers_names = set()

    providers_map = { _norm(p): p for p in providers }
    org_providers_map = { _norm(p): p for p in org_providers }
    comp_norm = list(providers_map.keys())
    cap_norm = list(org_providers_map.keys())

    def last_token(s):
        parts = _norm(s).split()
        return parts[-1] if parts else ""

    # simple blocking
    cap_by_last, cap_by_initial = {}, {}
    for n in cap_norm:
        lt = last_token(n)
        cap_by_last.setdefault(lt, []).append(n)
        cap_by_initial.setdefault(lt[:1], []).append(n)

    for cr in comp_norm:
        original_cr = providers_map[cr]
        if original_cr in matched_providers_set:
            continue

        best_name = None
        best_score = -1.0
        matched_here = False

        for current_threshold in range(90, threshold - 1, -5):
            lt = last_token(cr)
            cand = set(cap_by_last.get(lt, [])) | set(cap_by_initial.get(lt[:1], []))
            if not cand:
                cand = set(cap_norm)
            cand = [c for c in cand if c not in already_matched_org_providers_names]
            if not cand:
                continue

            best = process.extractOne(cr, cand, scorer=name_similarity)
            if best is None:
                continue

            match_name, score, _ = best
            score = float(score)

            if score > best_score:
                best_score = score
                best_name = match_name

            if score >= current_threshold:
                matched.append({
                    'providers': original_cr,
                    'org_providers_name': org_providers_map[match_name],
                    'score': score
                })
                matched_providers_set.add(original_cr)
                already_matched_org_providers_names.add(match_name)
                matched_here = True
                break

        if not matched_here:
            unmatched_map[cr] = {
                'providers': original_cr,
                'org_providers_name': org_providers_map[best_name] if best_name and best_score > 0 else None,
                'score': best_score if best_score > 0 else 0.0
            }

    unmatched = sorted(unmatched_map.values(), key=lambda x: x['score'], reverse=True)
    return {"matched": matched, "unmatched": unmatched}

def preprocess_PR_FDR(PR_df, FDR_df):
        # Merge first 2 rows into one header row
    header1 = PR_df.iloc[0].fillna("")
    header2 = PR_df.iloc[1].fillna("")

    # Build combined header
    new_cols = []
    for h1, h2 in zip(header1, header2):
        if h1 and h2:
            new_cols.append(f"{h1}_{h2}".strip())
        elif h1:  # only top header
            new_cols.append(h1.strip())
        else:     # only sub header
            new_cols.append(h2.strip())
    PR_df = PR_df[3:]
    PR_df.columns = new_cols
    PR_df = PR_df.reset_index(drop=True)

    # Optional: clean names
    PR_df.columns = (
        PR_df.columns.str.replace(r"\s+", "_", regex=True)
                 .str.replace(r"[^\w]", "", regex=True)
                 .str.replace("__", "_")
    )

    # take a copy of the slice to avoid SettingWithCopy
    PR_df = PR_df.loc[:, ["Personnel","Earnings_Reg","Gross"]].copy()
    PR_df.dropna(inplace=True)
    PR_df['Personnel'] = PR_df['Personnel'].str.split("\n").str[0]
    for row in PR_df.itertuples():
        personnel = row.Personnel
        personnel = personnel.strip().split(",")
        personnel = personnel[1].strip() + " " + personnel[0].strip()
        PR_df.at[row.Index, 'Personnel'] = personnel
    def _to_numeric_series(s: pd.Series) -> pd.Series:
        cleaned = (
            s.astype(str).str.strip()
             .str.replace(r'[,\s$]', '', regex=True)
             .str.replace(r'^\((.*)\)$', r'-\1', regex=True)
        )
        return pd.to_numeric(cleaned, errors='coerce')

    # Ensure numeric types for payroll
    PR_df["Earnings_Reg"] = _to_numeric_series(PR_df["Earnings_Reg"])
    PR_df["Gross"] = _to_numeric_series(PR_df["Gross"])

    # FDR cleanup and numeric (copy slice first)
    FDR_df = FDR_df.loc[:, ["PROVIDER","NET CHARGES","NET RECEIPTS"]].copy()
    FDR_df["NET CHARGES"] = _to_numeric_series(FDR_df["NET CHARGES"])
    FDR_df["NET RECEIPTS"] = _to_numeric_series(FDR_df["NET RECEIPTS"])

    FDR_df = FDR_df[~((FDR_df["NET CHARGES"] == 0.0) & (FDR_df["NET RECEIPTS"] == 0.0))]
    FDR_df.drop_duplicates(subset=["PROVIDER"], inplace=True)
    PR_df.drop_duplicates(subset=["Personnel"], inplace=True)
    return PR_df, FDR_df


def build_metadata(charge_capture_df, company_roster_df, PR_df, FDR_df):
    CC_used_col = ["Provider", "CPT Codes", "Charge Status"]
    CR_used_col = ['Name', "Manager", 'State/Region']

    CC_filtered_df = charge_capture_df[CC_used_col]
    CR_filtered_df = company_roster_df[CR_used_col]

    # Rename column safely
    CR_filtered_df = CR_filtered_df.rename(columns={'Name': 'Provider'})

    grouped_CC = CC_filtered_df.groupby('Provider')
    grouped_CR = CR_filtered_df.groupby('State/Region')
    cpt_pattern = re.compile(r'\b993\d{2}\b')  # Example pattern for 5-digit codes starting with 9930
    Margin_df = pd.DataFrame({
        'Clinician_Name': pd.Series(dtype='object'),
        'Manager': pd.Series(dtype='object'),
        'Gross_Consents': pd.Series(dtype='int'),
        'Gross_Encounters': pd.Series(dtype='int'),
        'Drafted_Encounters': pd.Series(dtype='int'), 
        'Collections': pd.Series(dtype='float'), 
        'Anticipated_Collections': pd.Series(dtype='float'),
        'Regular_Pay': pd.Series(dtype='float'),
        'Regular_Margin': pd.Series(dtype='float'),
        'Gross_Pay': pd.Series(dtype='float'),
        'Net_Margin': pd.Series(dtype='float'),
        'Anticipated_Net_Margin': pd.Series(dtype='float'),
        'region': pd.Series(dtype='object'),
    })
    for name, group in grouped_CC:
        group = group.copy()
        group['CPT Codes'] = group['CPT Codes'].astype(str).str.split(',')
        group = group.explode('CPT Codes')
        group['CPT Codes'] = group['CPT Codes'].str.strip()
        group['Charge Status'] = group['Charge Status'].str.strip().str.lower()
        # CCM Counts
        CCM_counts =group[group['CPT Codes']=="44444"]
        CCM_counts = CCM_counts['CPT Codes'].count()

        # Filter rows where 'CPT Codes' is in the target list and 'Charge Status' is 'draft'
        draft_counts = group[
            (group['CPT Codes'].str.match(cpt_pattern, na=False)) & 
            (group['Charge Status'] == 'draft')
        ]

        # Count the rows
        draft_counts = draft_counts['Charge Status'].count()
        # Gross Encounters    
        cpt_grouped = group[group['CPT Codes'].str.match(cpt_pattern, na=False)]
        gross_encounters = cpt_grouped['CPT Codes'].count()

        new_row = {'Clinician_Name': name,
                    'Manager': None,
                    'Gross_Consents': CCM_counts,
                    'Gross_Encounters': gross_encounters,
                    'Drafted_Encounters': draft_counts,
                    'Collections': 0.0,  # Placeholder for Collections
                    'Anticipated_Collections': 0.0,  # Placeholder for Anticipated Collections
                    'Regular_Pay': 0.0,  # Placeholder for Regular Pay
                    'Regular_Margin': 0.0,  # Placeholder for Regular Margin
                    'Gross_Pay': 0.0,  # Placeholder for Gross Pay
                    'Net_Margin': 0.0,  # Placeholder for Net Margin
                    'Anticipated_Net_Margin': 0.0,  # Placeholder for Anticipated Net Margin
                    "region": None,
                    }   

        Margin_df.loc[len(Margin_df)] = new_row
    
    
    PR_names = PR_df['Personnel'].tolist()
    FDR_names = FDR_df['PROVIDER'].tolist()
    Margin_df_names= Margin_df['Clinician_Name'].tolist()

    # Match providers from PR with Margin_df
    PR_result = match_providers(PR_names, Margin_df_names, threshold=85)
    matched_providers_PR = PR_result['matched']
    unmatched_providers_PR = PR_result['unmatched'] 
    print(f"Matched Providers from PR: {len(matched_providers_PR)}")
    print(f"Unmatched Providers from PR: {len(unmatched_providers_PR)}")
    # Update Margin_df with PR data
    for row in matched_providers_PR:
        clinician_name = row['providers']
        charge_capture_name = row['org_providers_name']
        score = row['score']

        reg_series = PR_df.loc[PR_df['Personnel'] == clinician_name, "Earnings_Reg"]
        reg_value = float(reg_series.iloc[0]) if not reg_series.empty and pd.notna(reg_series.iloc[0]) else np.nan
        Margin_df.loc[Margin_df['Clinician_Name'] == charge_capture_name, 'Regular_Pay'] = reg_value

        gross_series = PR_df.loc[PR_df['Personnel'] == clinician_name, "Gross"]
        gross_value = float(gross_series.iloc[0]) if not gross_series.empty and pd.notna(gross_series.iloc[0]) else np.nan
        Margin_df.loc[Margin_df['Clinician_Name'] == charge_capture_name, 'Gross_Pay'] = gross_value


    # Match providers from FDR with Margin_df
    FDR_result = match_providers(FDR_names, Margin_df_names, threshold=85)
    matched_providers_FDR = FDR_result['matched']
    unmatched_providers_FDR = FDR_result['unmatched']
    print(f"Matched Providers from FDR: {len(matched_providers_FDR)}")
    print(f"Unmatched Providers from FDR: {len(unmatched_providers_FDR)}")

    # Update Margin_df with FDR data
    for row in matched_providers_FDR:
        clinician_name = row['providers']
        charge_capture_name = row['org_providers_name']
        score = row['score']

        # Assign scalars, not Series
        charges_series = FDR_df.loc[FDR_df['PROVIDER'] == clinician_name, "NET CHARGES"]
        charges_value = float(charges_series.iloc[0]) if not charges_series.empty and pd.notna(charges_series.iloc[0]) else np.nan
        Margin_df.loc[Margin_df['Clinician_Name'] == charge_capture_name, 'Anticipated_Collections'] = charges_value

        receipts_series = FDR_df.loc[FDR_df['PROVIDER'] == clinician_name, "NET RECEIPTS"]
        receipts_value = float(receipts_series.iloc[0]) if not receipts_series.empty and pd.notna(receipts_series.iloc[0]) else np.nan
        Margin_df.loc[Margin_df['Clinician_Name'] == charge_capture_name, 'Collections'] = receipts_value
        # Regular_Margin "(regular pay column / collections)*100"
        if not pd.isna(receipts_value) and receipts_value != 0:
            regular_pay = Margin_df.loc[Margin_df['Clinician_Name'] == charge_capture_name, 'Regular_Pay'].values[0]
            if not pd.isna(regular_pay):
                regular_margin = (receipts_value / regular_pay ) * 100 if regular_pay != 0 else 0
                Margin_df.loc[Margin_df['Clinician_Name'] == charge_capture_name, 'Regular_Margin'] = round(regular_margin,2)
                
        # Net_Margin "(gross pay column / collections)*100"
        if not pd.isna(charges_value) and charges_value != 0:
            gross_pay = Margin_df.loc[Margin_df['Clinician_Name'] == charge_capture_name, 'Gross_Pay'].values[0]
            if not pd.isna(gross_pay):
                Anticipated_Net_Margin = (charges_value/ gross_pay ) * 100 if gross_pay != 0 else 0
                net_margin = (receipts_value / gross_pay) * 100 if gross_pay != 0 else 0
                Margin_df.loc[Margin_df['Clinician_Name'] == charge_capture_name, 'Anticipated_Net_Margin'] = round(Anticipated_Net_Margin, 2)
                Margin_df.loc[Margin_df['Clinician_Name'] == charge_capture_name, 'Net_Margin'] = round(net_margin, 2)

    

    unmatched_providers = []
    matched_providers = []
    Regional_Dashboard=pd.DataFrame(columns=["Region","RCS","RDO","Gross Encounters","Gross Consents","Gross Drafts","Regular_margin","Gross_margin","Anticipated_Net_Margin"])

    for name, group in grouped_CR:
        manager_group=group.groupby('Manager')
        for manager_name, manager_group in manager_group:
            Region_list=Margin_df['Clinician_Name']
            manager_group_list = manager_group['Provider']
            result=match_providers(manager_group_list, Region_list, threshold=85)
            unmatched_providers.extend(result['unmatched'])
            matched_providers.extend(result['matched'])
            matched_providers_CC = [item['org_providers_name'] for item in result['matched']]
            Region_Gross_Encounters = Margin_df[Margin_df['Clinician_Name'].isin([item['org_providers_name'] for item in result['matched']])]['Gross_Encounters'].sum()
            Region_Gross_Consents = Margin_df[Margin_df['Clinician_Name'].isin([item['org_providers_name'] for item in result['matched']])]['Gross_Consents'].sum()
            Region_Gross_Drafts = Margin_df[Margin_df['Clinician_Name'].isin([item['org_providers_name'] for item in result['matched']])]['Drafted_Encounters'].sum()
            Region_Regular_margin = Margin_df[Margin_df['Clinician_Name'].isin([item['org_providers_name'] for item in result['matched']])]['Regular_Margin'].mean()
            Region_Gross_margin = Margin_df[Margin_df['Clinician_Name'].isin([item['org_providers_name'] for item in result['matched']])]['Net_Margin'].mean()
            Region_Anticipated_Net_Margin = Margin_df[Margin_df['Clinician_Name'].isin([item['org_providers_name'] for item in result['matched']])]['Anticipated_Net_Margin'].mean()  
            # Create a new row for the Regional Dashboard
            print(matched_providers_CC)
            if not matched_providers_CC:
                continue  # Skip if no matched providers  
            new_row = {
                "Region": name,
                "RCS": manager_name,
                "RDO": matched_providers_CC,
                "Gross Encounters": Region_Gross_Encounters,
                "Gross Consents": Region_Gross_Consents,
                "Gross Drafts": Region_Gross_Drafts,
                "Regular_margin": Region_Regular_margin,  # Placeholder for Net Margin
                "Gross_margin": Region_Gross_margin,  # Placeholder for Gross Margin
                "Anticipated_Net_Margin": Region_Anticipated_Net_Margin,  # Placeholder for Anticipated Net Margin
            }
            Regional_Dashboard.loc[len(Regional_Dashboard)] = new_row
    Regional_Dashboard.columns = Regional_Dashboard.columns.str.replace(' ', '_')
    Margin_df.columns = Margin_df.columns.str.replace(' ', '_')
    for row in Regional_Dashboard.itertuples():
        region = row.Region
        Clinician_list = row.RDO
        manager= row.RCS    
        # Update the 'region' column for all clinicians in the list
        Margin_df.loc[Margin_df['Clinician_Name'].isin(Clinician_list), 'region'] = region
        Margin_df.loc[Margin_df['Clinician_Name'].isin(Clinician_list), 'Manager'] = manager

    
    # Drop rows with NaN values in the 'region' column
    Margin_df.dropna(subset=['region'], inplace=True)
    
    grouped_CC_ByRegion = Margin_df.groupby(['region', 'Manager'])
    return Regional_Dashboard, unmatched_providers, matched_providers, grouped_CC_ByRegion,Margin_df


PR_df = pd.read_excel(r"D:\payroll\PR_PayrollRegister_KYB_31 (2).xls",sheet_name="Payroll Register",header=None)
FDR_df = pd.read_excel(r"D:\payroll\Financial Dashboard Report by Provider as of August 11 2025, 7-39 AM.xlsx")
PR_df, FDR_df = preprocess_PR_FDR(PR_df, FDR_df)
charge_capture_df = pd.read_excel(r"D:\payroll\Charge_Capture_-_Jul_20th_-_Aug_2nd.xlsx")
company_roaster = pd.read_excel(r"D:\payroll\Company_Roster_1754677385.xlsx", skiprows=2)
Regional_Dashboard, unmatched_providers, matched_providers, grouped_CC_ByRegion,Margin_df = build_metadata(charge_capture_df, company_roaster,PR_df, FDR_df)

Matched Providers from PR: 69
Unmatched Providers from PR: 33
Matched Providers from FDR: 67
Unmatched Providers from FDR: 4
[]
[]
['Shnita Wiley', 'Chekeli Daniels', 'Mya Williams', 'Jessica Bonilla', 'Summer Williams', 'Cassandra Westbrook']
[]
[]
['Jillian Mills']
[]
[]
['Leah Gravesen', 'Kiryn Evans', 'Connie Zurski', 'Olumide Adeyemo', 'Miranda McMahan', 'Emily Hamilton', 'Christina Hay', 'Amit Parkhe', 'Sherry Boyd', 'Paige  Dennis', 'Sasha Moredock', 'Stephen Skimehorn']
['Aretha Johnson', 'Tamara  Baird', 'Jeanea Morris', 'Katelin Breuning', 'Ashley McPhillips', 'Brandi Cassaday']
[]
['Eli Perez', 'Tashica Bruce']
['Hina Patel', 'Roshana Brown']
['Adejimi Carrington', 'Vera Dankwah', 'Ahmad Almawaldi', 'Germaine Henderson', 'Nicole  Cartier', 'Melissa Townsend']
['Jeanne  Wilson']
['Kevin Widdis']
['Bobby Hill']
[]
['Shawanda Minor', 'Shawn Gorkiewicz', 'Kathryn  McAdam', 'Christopher Briggs', 'Anna OKeefe', 'Brenda Buchanan']
[]
['Abdulilah Obeid', 'Marina Memon', 'Hala Hojeij

In [ ]:
# Save Margin_df to Excel with autosized columns and wrapped text
output_path = r"D:\payroll\Margin_df.xlsx"

# Make a safe copy and stringify any list-like cells
_to_str = lambda v: ", ".join(map(str, v)) if isinstance(v, (list, tuple)) else v
_margin_out = Margin_df.copy()
for col in _margin_out.select_dtypes(include=["object"]).columns:
    _margin_out[col] = _margin_out[col].map(_to_str)

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    _margin_out.to_excel(writer, sheet_name="Margin", index=False)
    wb  = writer.book
    ws  = writer.sheets["Margin"]

    header_fmt = wb.add_format({"bold": True, "text_wrap": True, "valign": "top", "fg_color": "#DDEBF7", "border": 1})
    wrap_fmt   = wb.add_format({"text_wrap": True, "valign": "top"})
    int_fmt    = wb.add_format({"num_format": "#,##0"})
    float_fmt  = wb.add_format({"num_format": "#,##0.00"})

    # Rewrite headers with format
    for j, col in enumerate(_margin_out.columns):
        ws.write(0, j, col, header_fmt)

    # Autosize columns and set formats
    for j, col in enumerate(_margin_out.columns):
        ser = _margin_out[col]
        max_len = max(len(str(col)), int(ser.astype(str).str.len().quantile(0.95) if len(ser) else 0)) + 2
        width = min(max_len, 60)

        fmt = None
        if pd.api.types.is_integer_dtype(ser):
            fmt = int_fmt
        elif pd.api.types.is_float_dtype(ser):
            fmt = float_fmt
        else:
            fmt = wrap_fmt

        ws.set_column(j, j, width, fmt)

    # Freeze header and add autofilter
    ws.freeze_panes(1, 1)
    ws.autofilter(0, 0, len(_margin_out), len(_margin_out.columns) - 1)

In [2]:
Regional_Dashboard

,Region,RCS,RDO,Gross_Encounters,Gross_Consents,Gross_Drafts,Regular_margin,Gross_margin,Anticipated_Net_Margin
0,Georgia,Jillian Mills,"[Shnita Wiley, Chekeli Daniels, Mya Williams, ...",988,43,4,300.071667,0.000000,0.000000
1,Georgia,"Lee Wilford, Waris Hussain",[Jillian Mills],186,9,0,340.960000,0.000000,0.000000
2,Illinois,Eli Perez,"[Leah Gravesen, Kiryn Evans, Connie Zurski, Ol...",1579,86,24,143.282500,141.222500,438.291667
3,Illinois,Kevin Widdis,"[Aretha Johnson, Tamara Baird, Jeanea Morris,...",761,9,5,136.235000,129.176667,379.630000
4,Illinois,"Lee Wilford, Waris Hussain","[Eli Perez, Tashica Bruce]",96,2,0,57.110000,57.110000,166.330000
5,Illinois,Revana Yousif,"[Hina Patel, Roshana Brown]",0,0,0,0.000000,0.000000,0.000000
6,Illinois,Tashica Bruce,"[Adejimi Carrington, Vera Dankwah, Ahmad Almaw...",888,14,181,99.285000,99.285000,363.025000
7,Illinois,Waris Hussain,[Jeanne Wilson],141,8,0,224.570000,224.570000,564.820000
8,Illinois,"Waris Hussain, Lee Wilford",[Kevin Widdis],73,2,2,62.110000,62.110000,192.640000
9,Indiana,Christina Hay,[Bobby Hill],24,2,0,0.000000,0.000000,0.000000


In [3]:
for manger, group in grouped_CC_ByRegion:
    print(f"Manager: {manger}")
    print(group[['Clinician_Name', 'Gross_Encounters', 'Gross_Consents', 'Drafted_Encounters', 'Collections', 'Anticipated_Collections', 'Regular_Pay', 'Regular_Margin', 'Gross_Pay', 'Net_Margin', 'Anticipated_Net_Margin']])
    print("\n")

Manager: ('Georgia', 'Jillian Mills')
         Clinician_Name  Gross_Encounters  Gross_Consents  Drafted_Encounters  \
19  Cassandra Westbrook               130               0                   3   
22      Chekeli Daniels               187              18                   0   
40      Jessica Bonilla               259              18                   0   
55         Mya Williams               178               0                   0   
69         Shnita Wiley                77               3                   1   
72      Summer Williams               157               4                   0   

    Collections  Anticipated_Collections  Regular_Pay  Regular_Margin  \
19     12742.41                      0.0      5000.00          254.85   
22     17155.43                      0.0      4846.16          354.00   
40     24877.23                      0.0      4538.47          548.14   
55     13694.26                      0.0      4884.62          280.35   
69      6221.78              

In [337]:
FDR_df

,PROVIDER,NET CHARGES,NET RECEIPTS
1,ABDULILAH OBEID,14429.18,4164.58
2,Adam Geller,10996.35,6550.67
3,ADEJIMI CARRINGTON,20624.72,7452.90
4,Ahmad Almawaldi,19769.84,6561.13
5,Alan Tran,0.00,2997.81
...,...,...,...
71,Vera Appiah-dankwah,5666.07,1210.94
72,Victoria Francis,22762.19,6474.77
73,Wanalee Betts,0.00,4835.50
74,Waris Hussain,30426.59,10656.80
